### config

In [1]:


from collections import defaultdict
from vif.baselines.models import VerEvaluation,VerEval_Features
from vif.baselines.verifiers_baseline.ver_baseline import TexVerBaseline
from datasets.formatting.formatting import LazyBatch
from PIL import Image
import sys
from datasets import load_dataset, Dataset, load_from_disk,concatenate_datasets
from loguru import logger

from vif.utils.renderer.tex_renderer import TexRenderer
import openai
import os
from google import genai
from google.genai import types as genTypes
from vif.baselines.verifiers_baseline import (
    FalconVerifier,
    TextVerifier,
    TextVisualVerifier,
    ViperGPTVerifier,
    VisualPropertiesVerifier,
    VisualVerifier,
    VisualCodeVerifier,
    PicoBananaVerifier,
    ClipSimVerifier
)
import datasets
from tqdm import tqdm
import json

logger.remove()
logger.add(tqdm.write)

/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 14:07:59.590 | INFO     | vif.utils.caching:instantiate_cache:26 - seg_cache cache loaded


1

In [2]:

NUM_PROC = 1
client = openai.Client(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

gclient = genai.Client(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        http_options=genTypes.HttpOptions(api_version="v1alpha"),
    )

used_model = "qwen/qwen3-vl-8b-instruct"#"qwen/qwen3-vl-8b-thinking"

logger.configure(handlers=[{"sink": sys.stdout, "level": "WARNING"}])
ds = load_dataset("CharlyR/VeriTikz", "full", split="train")
ds = ds.select(range(0,25))
visual_verifier: TexVerBaseline = ViperGPTVerifier(
    model=used_model,
    temperature=0.5,
    client=client,
)

def custom_get_code(vereval):
    return open("notebooks/protos/code/d.py","r").read(),{}

#visual_verifier.get_code = custom_get_code


In [3]:
selected_row = ds[15]

In [4]:
selected_row

{'id': 'cat_blue_nose',
 'type': 'animal',
 'instruction': 'Make the nose of the cat blue',
 'code': '\\documentclass[tikz,border=5]{standalone}\n\\usepackage[prefix=]{xcolor-material}\n\n\\tikzset{\ncat/.pic={\n\\tikzset{x=3cm/5,y=3cm/5,shift={(0,-1/3)}}\n\\useasboundingbox (-1,-1) (1,2);\n\\fill [BlueGrey900] (0,-2)\n  .. controls ++(180:3) and ++(0:5/4) .. (-2,0)\n  arc (270:90:1/5)\n  .. controls ++(0:2) and ++(180:11/4) .. (0,-2+2/5);\n\\foreach \\i in {-1,1}\n  \\scoped[shift={(1/2*\\i,9/4)}, rotate=45*\\i]{\n    \\clip [overlay] (0, 5/9) ellipse [radius=8/9];\n    \\clip [overlay] (0,-5/9) ellipse [radius=8/9];\n    \\fill [BlueGrey900] ellipse [radius=1];\n    \\clip [overlay] (0, 7/9) ellipse [radius=10/11];\n    \\clip [overlay] (0,-7/9) ellipse [radius=10/11];\n    \\fill [Purple100] ellipse [radius=1];\n  };\n\\fill [BlueGrey900] ellipse [x radius=3/4, y radius=2];\n\\fill [BlueGrey100] ellipse [x radius=1/3, y radius=1];\n\\fill [BlueGrey900]\n  (0,15/8) ellipse [x radius=

### Exec

In [5]:
test_ver_input = VerEvaluation(
    id=selected_row["id"],
    approach_name="test_approach_code",
    config_metadata={"test":"test"},
    initial_code=selected_row["code"],
    initial_image=selected_row["original_image"],
    initial_instruction=selected_row["instruction"],
    initial_solution=selected_row["solution"],
    initial_solution_image=selected_row["solution_image"],
    expected=selected_row["expected"],
    theoretical_perfect_image=selected_row["theoretical_perfect_image"],
)

res_input:VerEvaluation = visual_verifier.assess_customization(test_ver_input)

2025-11-25 14:08:14.156 | WARNING  | vif.utils.detection_utils:log_and_append_token_data:336 - box detection worked.
2025-11-25 14:08:15.709 | WARNING  | vif.utils.detection_utils:log_and_append_token_data:336 - box detection worked.


In [6]:
res_input.errors

{}

In [7]:
print(res_input.additional_metadata["generated_function"])

def execute_command(initial_image, modified_image):
    # Create ImagePatch objects for both images
    initial_patch = ImagePatch(initial_image)
    modified_patch = ImagePatch(modified_image)
    
    # Find cat in the initial image
    cats_initial = initial_patch.find("cat")
    if not cats_initial:
        return False  # No cat found in the original image
    
    # Find cat in the modified image
    cats_modified = modified_patch.find("cat")
    if not cats_modified:
        return False  # No cat found in the modified image
    
    # Get the first cat from each image (assuming one cat per image)
    cat_initial = cats_initial[0]
    cat_modified = cats_modified[0]
    
    # Check if the cat's nose is blue in the modified image
    # We'll use the llm_query to ask about the color of the nose
    nose_color = cat_modified.simple_query("What color is the nose?")
    
    # If the nose is not blue, return False
    if nose_color.lower() != "blue":
        return False
    
    # 

In [9]:
res_input.additional_metadata

{'generated_function': 'def execute_command(initial_image, modified_image):\n    # Create ImagePatch objects for both images\n    initial_patch = ImagePatch(initial_image)\n    modified_patch = ImagePatch(modified_image)\n    \n    # Find cat in the initial image\n    cats_initial = initial_patch.find("cat")\n    if not cats_initial:\n        return False  # No cat found in the original image\n    \n    # Find cat in the modified image\n    cats_modified = modified_patch.find("cat")\n    if not cats_modified:\n        return False  # No cat found in the modified image\n    \n    # Get the first cat from each image (assuming one cat per image)\n    cat_initial = cats_initial[0]\n    cat_modified = cats_modified[0]\n    \n    # Check if the cat\'s nose is blue in the modified image\n    # We\'ll use the llm_query to ask about the color of the nose\n    nose_color = cat_modified.simple_query("What color is the nose?")\n    \n    # If the nose is not blue, return False\n    if nose_color.l